In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing as scale
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

We do the repeat experiment

In [2]:
input_file = ["Backdoor.csv" , "DDoS_HTTP.csv" , "DDoS_ICMP.csv" , "DDoS_TCP.csv" , "DDoS_UDP.csv"  , "Password.csv" , "Port_Scanning.csv" ,"Ransomware.csv", "SQL_injection.csv" , "Uploading.csv" , "Vulnerability_scanner.csv" ,"XSS.csv" ]

In [3]:
def pred(x, clfs, p_nodes, gms):
    p_x_node = np.zeros((len(x), len(clfs))) # p[x, in node i] = p(x | x in Node i) * p(x in Node i)
    p_y_given_node = np.zeros((len(x), len(clfs))) # p[y | x, node i]
    for i in range(len(clfs)):
        p_x_node[:, i] = p_nodes[i] * np.exp(gms[i].score_samples(x))
        p_y_given_node[:, i] = clfs[i].predict_proba(x)[:, 1]
    
    p_y = p_y_given_node * p_x_node / (np.sum(p_x_node, axis=-1, keepdims=True) + 1e-10)

    p_y = np.hstack([1-np.sum(p_y, axis=-1, keepdims=True), p_y])

    return np.argmax(p_y, axis=-1)

In [4]:
columns_to_drop_bi = ['Attack_type']

data_list = []
for i in range(len(input_file)):
    data = pd.read_csv(input_file[i])
    data['attack_type'] = i + 1
    data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
    data.drop(columns = columns_to_drop_bi, inplace = True)
    if len(data) < 2000:
        data_list.append(data)
    else:
        data0 = data[data['attack_type'].isin([0])]
        x1 = data0.sample(n = 1000, random_state = i + 42, axis = 0)
        data1 = data[data['attack_type'].isin([i+1])]
        x2 = data1.sample(n = 1000, random_state = i + 43, axis = 0)   
        data_selected = pd.concat([x1, x2], ignore_index=True)  
        data_list.append(data_selected) 
for i in range(len(data_list)):
    print(len(data_list[i]))  




2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000


In [5]:
data_list[9].tail()

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,http.content_length,http.response,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,...,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msgtype,mqtt.proto_len,mqtt.topic_len,mqtt.ver,Attack_label,attack_type
1995,0,0,0,0,0,0,150,1934832499,19721,0,...,0,0,0,0,0,0,0,0,1,10
1996,0,0,0,0,0,0,0,0,24507,0,...,0,0,0,0,0,0,0,0,1,10
1997,0,0,0,0,0,0,1,3703765364,46928,0,...,0,0,0,0,0,0,0,0,1,10
1998,0,0,0,0,0,0,1,208620284,44688,0,...,0,0,0,0,0,0,0,0,1,10
1999,0,0,0,0,0,0,1,3557485597,42863,0,...,0,0,0,0,0,0,0,0,1,10


In [6]:
rs = 42 # set the random state

In [7]:
from sklearn.svm import SVC
clfs = [] # This is going to contain 14 different classifiers
n_samples = []
x_train_list = []
y_train_list = []
x_test_list = []
y_test_list = []
y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
for i in tqdm(range(len(input_file))): # reading the data
    data0 = data_list[i]
    y[i][data0.attack_type == 0] = 0
    n_samples.append(len(y[i])) # the number of this node
    x = data_list[i]
    x.drop(columns='attack_type')
    x.drop(columns='Attack_label')
    x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = rs)
    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_test_list.append(x_test)
    y_test_list.append(y_test)

x_train_full = pd.concat(x_train_list, ignore_index=True)
scaler = scale.StandardScaler().fit(x_train_full)

for i in range(len(input_file)):
    x_train_list[i] = scaler.transform(x_train_list[i])

for i in tqdm(range(len(input_file))):
    #classifier = SVC(kernel = 'rbf', random_state = 41, gamma='scale',max_iter=-1, probability=True)
    # classifier = SVC(kernel = 'rbf', random_state = rs, gamma='scale', max_iter=-1, probability=True)
    # classifier.fit(data_list[i], y[i])
    classifier = RandomForestClassifier()
    classifier.fit(x_train_list[i], y_train_list[i])
    clfs.append(classifier)

total_n_samples = np.sum(n_samples)
print('total number is ',total_n_samples)
p_nodes =  np.array(n_samples) / total_n_samples
print('The proportion of the nodes are:', p_nodes)



total number is  24000
The proportion of the nodes are: [0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333]


In [8]:
for i in range(len(input_file)):
    x_test_list[i] = scaler.transform(x_test_list[i])

In [9]:
x_test = np.vstack(x_test_list)
y_test = np.hstack(y_test_list)
x_train = np.vstack(x_train_list)
y_train = np.hstack(y_train_list)

In [10]:
# GMM
from sklearn.mixture import GaussianMixture
gms = []
K = 15 # The number of the components of the GMM, one could change it by analyzing the graph
for i in tqdm(range(len(input_file))):
    x = x_train_list[i]
    gm = GaussianMixture(n_components = K).fit(x)  
    gms.append(gm)

In [12]:

prediction = pred(x_test, clfs, p_nodes, gms)

In [13]:

correct = prediction == y_test
accuracy = np.mean(correct)
print('Overall accuracy is:', accuracy)

Overall accuracy is: 0.9991666666666666


In [14]:
accs = []
for i in range(len(input_file)+1):
    if len(correct[y_test==i]) == 0:
        accs.append(0)
    else:
        accs.append(np.mean(correct[y_test==i]))
print('Accuracies:', accs)
recalls = []
for i in range(len(input_file)+1):
    if len(correct[prediction==i]) == 0:
        recalls.append(0)
    else:
        recalls.append(np.mean(correct[prediction==i]))
print('Recalls:', recalls)

Accuracies: [1.0, 1.0, 0.9958847736625515, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9876543209876543, 0.9958847736625515]
Recalls: [0.9983813531887342, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


Repeat the test:

In [5]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1 
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x.drop(columns='attack_type')
        x.drop(columns='Attack_label')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)

    for i in range(len(input_file)):
        x_train_list[i] = scaler.transform(x_train_list[i])

    for i in range(len(input_file)):
        #classifier = LogisticRegression(random_state=epoch)
        classifier = KNeighborsClassifier()
        classifier.fit(x_train_list[i], y_train_list[i])
        clfs.append(classifier)

    total_n_samples = np.sum(n_samples)
    p_nodes =  np.array(n_samples) / total_n_samples


    # Test:
    for i in range(len(input_file)):
        x_test_list[i] = scaler.transform(x_test_list[i])

    x_test = np.vstack(x_test_list)
    y_test = np.hstack(y_test_list)
    x_train = np.vstack(x_train_list)
    y_train = np.hstack(y_train_list)


    from sklearn.mixture import GaussianMixture
    gms = []
    K = 15 # The number of the components of the GMM, one could change it by analyzing the graph
    for i in range(len(input_file)):
        x = x_train_list[i]
        gm = GaussianMixture(n_components = K).fit(x)  
        gms.append(gm)

    prediction = pred(x_test, clfs, p_nodes, gms)
    correct = prediction == y_test

    accs = []
    for i in range(len(input_file)+1):
        if len(correct[y_test==i]) == 0:
            accs.append(0)
        else:
            accs.append(np.mean(correct[y_test==i]))
    
    recall[epoch,:] = accs
      

    recalls = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            recalls.append(0)
        else:
            recalls.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = recalls
    




for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))




0 Precision mean= 0.9987491367325021
0 Precision std= 0.0007168110173182934
1 Precision mean= 0.9980049114476872
1 Precision std= 0.00456138534379315
2 Precision mean= 1.0
2 Precision std= 0.0
3 Precision mean= 1.0
3 Precision std= 0.0
4 Precision mean= 1.0
4 Precision std= 0.0
5 Precision mean= 0.9942352697017842
5 Precision std= 0.014049899407270689
6 Precision mean= 1.0
6 Precision std= 0.0
7 Precision mean= 1.0
7 Precision std= 0.0
8 Precision mean= 0.9998000000000001
8 Precision std= 0.0008717797887081355
9 Precision mean= 1.0
9 Precision std= 0.0
10 Precision mean= 1.0
10 Precision std= 0.0
11 Precision mean= 1.0
11 Precision std= 0.0
12 Precision mean= 1.0
12 Precision std= 0.0
Recall mean of  0  = 0.9993186073759459
Recall std of  0  = 0.0014446323326059612
Recall mean of  1  = 0.9998154981549815
Recall std of  1  = 0.0008042248973322168
Recall mean of  2  = 0.9961963776448831
Recall std of  2  = 0.004762947398046391
Recall mean of  3  = 1.0
Recall std of  3  = 0.0
Recall mean

Full:

Random Forest

In [6]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x.drop(columns='attack_type')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = LogisticRegression(random_state = epoch)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))



/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo


0 Precision mean= 1.0
0 Precision std= 0.0
1 Precision mean= 0.990442188321613
1 Precision std= 0.009441524909574115
2 Precision mean= 0.993199725013859
2 Precision std= 0.00652531463590949
3 Precision mean= 0.9986139223544572
3 Precision std= 0.0028868905337346294
4 Precision mean= 0.9967121489684914
4 Precision std= 0.004257425642298421
5 Precision mean= 0.9982319551681963
5 Precision std= 0.0038521438365888444
6 Precision mean= 0.988618252823995
6 Precision std= 0.009381431594957931
7 Precision mean= 0.9093605023900228
7 Precision std= 0.02175382086758046
8 Precision mean= 0.8819766258653983
8 Precision std= 0.018838060572445543
9 Precision mean= 0.9528630738500702
9 Precision std= 0.034568072834674876
10 Precision mean= 0.8808136183445214
10 Precision std= 0.031247136662436076
11 Precision mean= 0.9350358907289085
11 Precision std= 0.024998532463082656
12 Precision mean= 0.9046096937180306
12 Precision std= 0.02751995038132959
Recall mean of  0  = 0.9999670619235836
Recall std of 

/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM_Full

In [17]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x.drop(columns='attack_type')
        x.drop(columns='Attack_label')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = SVC(kernel = 'rbf', gamma='scale',max_iter=-1, probability=True)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))




0 Precision mean= 0.999984399375975
0 Precision std= 6.800154358096656e-05
1 Precision mean= 0.9976529648724457
1 Precision std= 0.0031631506966908443
2 Precision mean= 0.965556772460076
2 Precision std= 0.007356644844461791
3 Precision mean= 0.9972321211924602
3 Precision std= 0.002825281657089061
4 Precision mean= 0.9912792684879147
4 Precision std= 0.009627056439872949
5 Precision mean= 0.9895729633941311
5 Precision std= 0.011034833193231203
6 Precision mean= 0.9682193232693311
6 Precision std= 0.010985600822172432
7 Precision mean= 0.9004940224928726
7 Precision std= 0.026367215887007512
8 Precision mean= 0.9453368874215655
8 Precision std= 0.020426057223423884
9 Precision mean= 0.8620218018595684
9 Precision std= 0.03530138198903517
10 Precision mean= 0.817008731203266
10 Precision std= 0.024687076343400154
11 Precision mean= 0.888968799300286
11 Precision std= 0.026869232499324158
12 Precision mean= 0.9471486421612425
12 Precision std= 0.02471964580277541
Recall mean of  0  = 0

LR:

In [15]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x.drop(columns='attack_type')
        x.drop(columns='Attack_label')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = LogisticRegression(random_state = rs+1)
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))



/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo


0 Precision mean= 1.0
0 Precision std= 0.0
1 Precision mean= 0.990442188321613
1 Precision std= 0.009441524909574115
2 Precision mean= 0.993199725013859
2 Precision std= 0.00652531463590949
3 Precision mean= 0.9986139223544572
3 Precision std= 0.0028868905337346294
4 Precision mean= 0.9967121489684914
4 Precision std= 0.004257425642298421
5 Precision mean= 0.9982319551681963
5 Precision std= 0.0038521438365888444
6 Precision mean= 0.988618252823995
6 Precision std= 0.009381431594957931
7 Precision mean= 0.9093605023900228
7 Precision std= 0.02175382086758046
8 Precision mean= 0.8819766258653983
8 Precision std= 0.018838060572445543
9 Precision mean= 0.9528630738500702
9 Precision std= 0.034568072834674876
10 Precision mean= 0.8808136183445214
10 Precision std= 0.031247136662436076
11 Precision mean= 0.9350358907289085
11 Precision std= 0.024998532463082656
12 Precision mean= 0.9046096937180306
12 Precision std= 0.02751995038132959
Recall mean of  0  = 0.9999670619235836
Recall std of 

/Users/irohayachiyo/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
num_epoch = 20

recall , precision = np.ones((num_epoch,len(input_file)+1))*0 , np.ones((num_epoch,len(input_file)+1))*0

for epoch in tqdm(range(num_epoch)):
    #Data Process
    columns_to_drop_bi = ['Attack_type']

    data_list = []
    for i in range(len(input_file)):
        data = pd.read_csv(input_file[i])
        data['attack_type'] = i + 1
        data.loc[data.Attack_type == 'Normal', 'attack_type'] = 0
        data.drop(columns = columns_to_drop_bi, inplace = True)
        if len(data) < 2000:
            data_list.append(data)
        else:
            data0 = data[data['attack_type'].isin([0])]
            x1 = data0.sample(n = 1000, random_state = i + epoch, axis = 0)
            data1 = data[data['attack_type'].isin([i+1])]
            x2 = data1.sample(n = 1000, random_state = i + epoch + 1, axis = 0)   
            data_selected = pd.concat([x1, x2], ignore_index=True)  
            data_list.append(data_selected)  
    #Training


    clfs = [] # This is going to contain 12 different classifiers
    n_samples = []
    x_train_list = []
    y_train_list = []
    x_test_list = []
    y_test_list = []
    y = [np.ones(len(data_list[i])) * (i + 1) for i in range(len(data_list))]
    for i in range(len(input_file)): # reading the data
        data0 = data_list[i]
        y[i][data0.attack_type == 0] = 0
        n_samples.append(len(y[i])) # the number of this node
        x = data_list[i]
        x.drop(columns='attack_type')
        x.drop(columns='Attack_label')
        x_train, x_test, y_train, y_test = train_test_split(x, y[i], test_size = 0.25, random_state = epoch+40)
        x_train_list.append(x_train)
        y_train_list.append(y_train)
        x_test_list.append(x_test)
        y_test_list.append(y_test)

    x_train_full = pd.concat(x_train_list, ignore_index=True)
    y_train_full = np.hstack(y_train_list)
    x_test_full = pd.concat(x_test_list, ignore_index=True)
    #y_test_full = pd.concat(y_test_list, ignore_index=True)
    scaler = scale.StandardScaler().fit(x_train_full)
    x_train_full = scaler.transform(x_train_full)
    classifier = KNeighborsClassifier()
    classifier.fit(x_train_full, y_train_full)

    x_test_full = scaler.transform(x_test_full)

    #x_test = np.vstack(x_test_list)
    y_test_full = np.hstack(y_test_list)
   # x_train = np.vstack(x_train_list)
    


    prediction = classifier.predict(x_test_full)
    correct = prediction == y_test_full

    rec = []
    for i in range(len(input_file)+1):
        if len(correct[y_test_full==i]) == 0:
            rec.append(0)
        else:
            rec.append(np.mean(correct[y_test_full==i]))
    
    recall[epoch,:] = rec
      

    pr = []
    for i in range(len(input_file)+1):
        if len(correct[prediction==i]) == 0:
            pr.append(0)
        else:
            pr.append(np.mean(correct[prediction==i]))
    
    precision[epoch,:] = pr
    
for i in range(len(input_file)+1):
      print(i,'Precision mean=',np.mean(precision[:,i]))
      print(i,'Precision std=',np.std(precision[:,i]))

for i in range(len(input_file)+1):
      print('Recall mean of ',i,' =',np.mean(recall[:,i]))
      print('Recall std of ',i,' =',np.std(recall[:,i]))




0 Precision mean= 1.0
0 Precision std= 0.0
1 Precision mean= 0.9656997199487243
1 Precision std= 0.011757366302334584
2 Precision mean= 0.9862230263093192
2 Precision std= 0.007011488430508363
3 Precision mean= 0.9966538840260268
3 Precision std= 0.003579273615586777
4 Precision mean= 0.9932482439185814
4 Precision std= 0.00521235321780772
5 Precision mean= 1.0
5 Precision std= 0.0
6 Precision mean= 0.9785019613935922
6 Precision std= 0.010082666904614462
7 Precision mean= 0.9677889154493101
7 Precision std= 0.00772782180136017
8 Precision mean= 0.9537367457063086
8 Precision std= 0.011461444910340707
9 Precision mean= 0.8533055560214062
9 Precision std= 0.024128831149708495
10 Precision mean= 0.8746801783508842
10 Precision std= 0.01903147619286737
11 Precision mean= 0.8998460694256879
11 Precision std= 0.025524724010747273
12 Precision mean= 0.9329077493951395
12 Precision std= 0.020335162283407034
Recall mean of  0  = 0.9998842057957733
Recall std of  0  = 0.00015792424276320249
Re